In [2]:
!pip install osmium geopandas matplotlib contextily

  Using cached matplotlib-3.10.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.2 kB)
  Using cached pillow-11.2.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.9 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached rasterio-1.4.3-cp313-cp313-macosx_14_0_arm64.whl.metadata (9.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached cligj-0.7.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl.metadata (6.4 kB)


In [ ]:
import osmium
import geopandas
import matplotlib.pyplot as plt
import contextily as cx
import os

# Download a sample PBF file
# For this example, let's assume 'liechtenstein-latest.osm.pbf' is in the current directory.
# You can download it from: https://download.geofabrik.de/europe/liechtenstein-latest.osm.pbf
pbf_file = 'germany-latest.osm.pbf'

if not os.path.exists(pbf_file):
    print(f"Error: {pbf_file} not found. Please download it first.")
    # You might add code here to automatically download for demonstration
    # e.g., using requests library, but for simplicity, assuming manual download.
    exit()

print(f"Reading {pbf_file}...")

# 1. Use osmium.FileProcessor with GeoInterfaceFilter
# with_locations() is crucial for getting coordinates for ways/areas
# with_areas() correctly handles closed ways and multipolygon relations for polygons
fp = osmium.FileProcessor(pbf_file) \
    .with_locations() \
    .with_areas() \
    .with_filter(osmium.filter.GeoInterfaceFilter())

# 2. Convert to GeoDataFrame
# The GeoInterfaceFilter makes osmium.FileProcessor iterable with geo_interface compliant objects
# You can filter by type here to get specific features (e.g., 'way' for roads, 'area' for buildings)
# For a simple map, we'll collect all geoj_objects and then filter.
geoj_objects = []
for obj in fp:
    if obj.is_node() or obj.is_way() or obj.is_area():
        # Only include objects that have a geometry (i.e., not just tags)
        if '__geo_interface__' in dir(obj):
            geoj_objects.append(obj)

# Create a GeoDataFrame from the collected objects
# Filter for geometry types relevant to plotting (points, lines, polygons)
# Note: This approach collects all features first, which can be memory intensive for large files.
# For very large files, consider iterating and appending to lists for nodes, ways, areas separately
# or using pyrosm as it's optimized for extracting common features.
# For this example, let's just plot all geometries.
try:
    gdf = geopandas.GeoDataFrame.from_features(geoj_objects, crs="EPSG:4326")
    print(f"Loaded {len(gdf)} features into GeoDataFrame.")

    # Filter out potential empty geometries or non-spatial objects if any
    gdf = gdf[~gdf.geometry.is_empty]

    if gdf.empty:
        print("No valid geometries found to plot.")
    else:
        # Separate data by geometry type for different plotting styles
        points = gdf[gdf.geometry.geom_type == 'Point']
        lines = gdf[gdf.geometry.geom_type == 'LineString']
        polygons = gdf[gdf.geometry.geom_type == 'Polygon']
        multi_polygons = gdf[gdf.geometry.geom_type == 'MultiPolygon']
        # Handle MultiLineString if present
        multi_lines = gdf[gdf.geometry.geom_type == 'MultiLineString']
        lines = geopandas.GeoDataFrame(pd.concat([lines, multi_lines], ignore_index=True), crs=gdf.crs)


        # 3. Plot the data
        fig, ax = plt.subplots(1, 1, figsize=(10, 10))

        # Plot polygons (e.g., buildings, landuse)
        if not polygons.empty:
            polygons.plot(ax=ax, color='lightgray', edgecolor='darkgray', linewidth=0.5, label='Polygons')
        if not multi_polygons.empty:
            multi_polygons.plot(ax=ax, color='lightgray', edgecolor='darkgray', linewidth=0.5, label='Polygons')

        # Plot lines (e.g., roads, paths)
        if not lines.empty:
            lines.plot(ax=ax, color='blue', linewidth=1, label='Lines')

        # Plot points (e.g., POIs, nodes) - consider plotting only significant points or sampling
        if not points.empty:
            points.plot(ax=ax, color='red', markersize=5, label='Points', zorder=5) # zorder to ensure points are on top

        # Add a basemap for context (requires `contextily`)
        # Ensure your GeoDataFrame is in Web Mercator (EPSG:3857) for best basemap results
        # OSM data is typically in WGS84 (EPSG:4326), so reproject for basemap
        if not gdf.empty:
            gdf_webmercator = gdf.to_crs(epsg=3857)
            cx.add_basemap(ax, crs=gdf_webmercator.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)
            ax.set_title(f"OSM Data from {pbf_file}")
            ax.set_axis_off() # Turn off axes for a cleaner map look
            plt.legend()
            plt.show()
        else:
            print("No geometries to reproject for basemap.")

except Exception as e:
    print(f"An error occurred during processing or plotting: {e}")
    print("Ensure you have a valid PBF file and necessary libraries installed.")

Reading germany-latest.osm.pbf...
